In [ ]:
%run "0_Workspace_setup.ipynb"

## Introduction
This notebook runs the NHM domain model using `pywatershed`.

## Prepare NHM domain for pywatershed run

### Make pywatershed .nc input files from NHM domain input file (cbh.nc).

In [ ]:
pws_prcp_input_file = model_dir / "prcp.nc"
pws_tmin_input_file = model_dir / "tmin.nc"
pws_tmax_input_file = model_dir / "tmax.nc"
nhmx_input_file = model_dir / "cbh.nc"
input_file_path_list = [pws_prcp_input_file, pws_tmin_input_file, pws_tmax_input_file]

for input_file_path in input_file_path_list:
    if not input_file_path.exists():
        con.print(
            f"One or more of the pywatershed input files does not exist. All input file will be rewritten from the cbh.nc file."
        )
        with xr.open_dataset(
            nhmx_input_file
        ) as input:  # This is the input file given with NHMx
            model_input = input.swap_dims({"nhru": "nhm_id"}).drop("nhru")
            # model_input = input.rename({'hru-d':'hruid'})
            prcp = getattr(model_input, "prcp")
            tmin = getattr(model_input, "tmin")
            tmax = getattr(model_input, "tmax")
        prcp.to_netcdf(pws_prcp_input_file)
        tmin.to_netcdf(pws_tmin_input_file)
        tmax.to_netcdf(pws_tmax_input_file)
        con.print(
            f"The pywatershed input file [bold]{pl.Path(input_file_path).stem}[/bold] was missing. All pywatershed input files were created in {model_dir} from the cbh.nc file."
        )
    else:
        pass
con.print(
    f"[bold][green]Optional:[/bold][/green] To recreate pywatershed input files in {model_dir}, delete [bold]prcp.nc[/bold], [bold]tmin.nc[/bold], and [bold]tmax.nc[/bold] files and re-run this notebook."
)

### Parameter file check
pywatershed  requires the siolzone variable "pref_flow_infil_frac" to be present where as PRMS does not. If the variable is not in the PRMS files we must add it to the parmaeter as all zeros before passing the parameters to the model.

In [ ]:
params = pws.parameters.PrmsParameters.load(param_filename)
if not "pref_flow_infil_frac" in params.parameters.keys():
    # Parameter objects are not directly editable in pywatershed,
    # so we export to an equivalent object we can edit, in this case
    # an xarray dataset, then we convert back
    params_ds = params.to_xr_ds()
    params_ds["pref_flow_infil_frac"] = params_ds.pref_flow_den[:] * 0.0
    params = pws.parameters.PrmsParameters.from_ds(params_ds)

### Custom Run for NHM domain model
The custom run loop will output the pywatershed standard output variables only and outputs each variable as a .nc file. The standard output variables, `selected_output_variables`, were selected in [notebook 0](.\0_Workspace_setup.ipynb).

In [ ]:
%%time
control = pws.Control.load_prms(
    model_dir / control_file_name, warn_unused_options=False
)
# Sets control options for both cases
control.options = control.options | {
    "input_dir": model_dir,
    "budget_type": None,
    "verbosity": 0,
    "calc_method": "numba",
}

control.options = control.options | {
    "netcdf_output_var_names": selected_output_variables,
    "netcdf_output_dir": out_dir,
}

model = pws.Model(
    [
        pws.PRMSSolarGeometry,
        pws.PRMSAtmosphere,
        pws.PRMSCanopy,
        pws.PRMSSnow,
        pws.PRMSRunoff,
        pws.PRMSSoilzone,
        pws.PRMSGroundwater,
        pws.PRMSChannel,
    ],
    control=control,
    parameters=params,
)

model.run()

### Create custom output variables from standard output variables.

In [ ]:
hru_streamflow_out = sum(
    xr.load_dataarray(f"{out_dir / ff}.nc")
    for ff in ["sroff_vol", "ssres_flow_vol", "gwres_flow_vol"]
)
hru_streamflow_out.to_netcdf(out_dir / "hru_streamflow_out.nc")
del hru_streamflow_out

### Filter `seg_outflow` for only segments that have gages

In [ ]:
# For streamflow, just keep output on the POIs.
# - 1 is related to the indexing in fortran; made a a tuple see above
wh_gages = (params.parameters["poi_gage_segment"] - 1,)
for var in ["seg_outflow"]:
    data = xr.load_dataarray(f"{out_dir / var}.nc")[:, wh_gages[0]]
    data = data.assign_coords(npoi_gages=("nhm_seg", params.parameters["poi_gage_id"]))
    out_file = f"{out_dir / var}.nc"
    data.to_netcdf(out_file)
    del data